<a href="https://colab.research.google.com/github/AnalyticsKnight/yemoonsaBigdata/blob/dev/src/P2_C4_01_MotorTrendCarRoadTests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 4. A to Z 빅데이터 분석 실습  

## 1.실습 A to Z – Motor Trend Car Road Tests

### mtcars: Motor Trend Car Road Tests
- https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv

In [1]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata/

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 261, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (246/246), done.
remote: Total 261 (delta 130), reused 24 (delta 9), pack-reused 0
Receiving objects: 100% (261/261), 3.47 MiB | 5.01 MiB/s, done.
Resolving deltas: 100% (130/130), done.


### (2) 사전준비

#### 412-01

In [2]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
X_test = pd.read_csv("./yemoonsaBigdata/datasets/Part2/mpg_X_test.csv")
X_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/mpg_X_train.csv")
y_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/mpg_y_train.csv")

# 사용자 코딩

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'gender': pred}).to_csv('003000000.csv', index=False)

#### 412-02

In [3]:
import pandas as pd
import numpy as np

#### 412-03

In [4]:
X_test = pd.read_csv("./yemoonsaBigdata/datasets/Part2/mpg_X_test.csv")
X_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/mpg_X_train.csv")
y_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/mpg_y_train.csv")

### (3) 데이터 전처리

#### 413-01

In [5]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          278 non-null    object 
 1   mpg           278 non-null    float64
 2   cylinders     278 non-null    int64  
 3   displacement  278 non-null    float64
 4   horsepower    274 non-null    float64
 5   weight        278 non-null    int64  
 6   acceleration  278 non-null    float64
 7   model_year    278 non-null    int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 17.5+ KB
None


#### 413-02

In [6]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train[['horsepower']] = imputer.fit_transform( X_train[['horsepower']] )
X_test[['horsepower']] = imputer.fit_transform( X_test[['horsepower']] )

#### 413-03

In [7]:
print(X_train.describe())

              mpg   cylinders  displacement  horsepower       weight  \
count  278.000000  278.000000    278.000000  278.000000   278.000000   
mean    23.732734    5.374101    189.994604  103.383212  2948.464029   
std      7.647295    1.677084    105.471423   38.695458   862.949746   
min     10.000000    3.000000     68.000000   46.000000  1613.000000   
25%     18.000000    4.000000     98.000000   75.000000  2206.250000   
50%     23.000000    4.000000    140.500000   90.500000  2737.500000   
75%     29.000000    6.000000    258.000000  118.750000  3560.000000   
max     46.600000    8.000000    455.000000  230.000000  5140.000000   

       acceleration  model_year  
count    278.000000  278.000000  
mean      15.580216   76.057554  
std        2.745907    3.605591  
min        8.000000   70.000000  
25%       14.000000   73.000000  
50%       15.500000   76.000000  
75%       17.000000   79.000000  
max       24.800000   82.000000  


#### 413-04

In [8]:
COL_DEL = ['name']
COL_NUM = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year']
COL_CAT = []
COL_Y = ['isUSA']

X_train = X_train.iloc[:, 1:]
X_test = X_test.iloc[:, 1:]

### (4) 데이터 모형 구축

#### 414-01

In [9]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3)

#### 414-02

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr[COL_NUM])
X_tr[COL_NUM]=scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM]=scaler.transform(X_val[COL_NUM])
X_test[COL_NUM]=scaler.transform(X_test[COL_NUM])

#### 414-03

In [11]:
from sklearn.neighbors import KNeighborsClassifier
modelKNN = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
modelKNN.fit(X_tr, y_tr.values.ravel())

from sklearn.tree import DecisionTreeClassifier
modelDT = DecisionTreeClassifier(max_depth=10)
modelDT.fit(X_tr, y_tr)

DecisionTreeClassifier(max_depth=10)

### (5) 데이터 모형 평가

#### 415-01

In [12]:
y_val_pred = modelKNN.predict(X_val)

y_val_pred_probaKNN = modelKNN.predict_proba(X_val)
y_val_pred_probaDT = modelDT.predict_proba(X_val)

#### 415-02

In [13]:
from sklearn.metrics import roc_auc_score

scoreKNN = roc_auc_score(y_val, y_val_pred_probaKNN[:, 1])
scoreDT = roc_auc_score(y_val, y_val_pred_probaDT[:, 1])

print( scoreKNN, scoreDT )

0.9256559766763848 0.8632653061224489


#### 415-03

In [14]:
best_model = None
best_score = 0

for i in range(2, 10):
    model = KNeighborsClassifier(n_neighbors=i, metric='euclidean')
    model.fit(X_tr, y_tr.values.ravel())
    y_val_pred_proba = model.predict_proba(X_val)
    score = roc_auc_score(y_val, y_val_pred_proba[:, 1])
    print(i,"개의 이웃 확인 : ", score)
    if best_score <= score:
        best_model = model

2 개의 이웃 확인 :  0.8994169096209913
3 개의 이웃 확인 :  0.9227405247813412
4 개의 이웃 확인 :  0.929737609329446
5 개의 이웃 확인 :  0.9256559766763848
6 개의 이웃 확인 :  0.9361516034985423
7 개의 이웃 확인 :  0.930320699708455
8 개의 이웃 확인 :  0.9376093294460641
9 개의 이웃 확인 :  0.9472303206997085


#### 415-04

In [15]:
best_model.predict_proba(X_test)

array([[0.55555556, 0.44444444],
       [0.33333333, 0.66666667],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.55555556, 0.44444444],
       [0.88888889, 0.11111111],
       [0.66666667, 0.33333333],
       [0.77777778, 0.22222222],
       [0.66666667, 0.33333333],
       [0.55555556, 0.44444444],
       [0.        , 1.        ],
       [0.44444444, 0.55555556],
       [0.        , 1.        ],
       [0.44444444, 0.55555556],
       [0.77777778, 0.22222222],
       [0.77777778, 0.22222222],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.77777778, 0.22222222],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.33333333, 0.66666667],
       [0.77777778, 0.22222222],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.44444444, 0.55555556],
       [0.

#### 415-05

In [16]:
pred = best_model.predict_proba(X_test)[:,1]
print(pred)

[0.44444444 0.66666667 1.         1.         0.44444444 0.11111111
 0.33333333 0.22222222 0.33333333 0.44444444 1.         0.55555556
 1.         0.55555556 0.22222222 0.22222222 1.         1.
 0.22222222 1.         1.         1.         1.         1.
 1.         0.66666667 0.22222222 1.         1.         0.55555556
 0.11111111 0.11111111 0.66666667 0.77777778 1.         0.66666667
 0.55555556 1.         0.22222222 0.77777778 1.         0.11111111
 0.11111111 0.55555556 0.44444444 0.44444444 0.33333333 1.
 0.33333333 1.         0.         0.22222222 1.         0.22222222
 1.         1.         1.         0.88888889 1.         0.44444444
 0.88888889 0.66666667 1.         1.         1.         1.
 1.         0.22222222 0.77777778 0.11111111 0.88888889 1.
 0.44444444 0.11111111 1.         0.22222222 1.         0.22222222
 0.22222222 1.         1.         1.         0.11111111 1.
 1.         1.         0.22222222 0.22222222 0.33333333 1.
 1.         1.         1.         1.         1.    

#### 415-06

In [17]:
pd.DataFrame({'isUSA': pred}).to_csv('./yemoonsaBigdata/res/003000000.csv', index=False)